# App

In [ ]:
from app import app

In [ ]:
app.run_server(debug=True, port=8000)

# Data
*create data for pages*

In [3]:
import pandas as pd
import os

In [4]:
os.chdir('/home/lbk/works/pf')

In [5]:
from pf_utils import PortfolioManager as PM 

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [6]:
path_data = 'data'
path_tran = 'transaction'

In [7]:
file_cost='transaction_cost'

## Price

In [8]:
universe = 'UV_FUND'
fund = PM.create_universe(universe)
cost = PM.get_cost(universe, file=file_cost, path=path_tran)
fee = cost['fee']

224 securities from 2005-01-31 to 2025-01-31 uploaded.
REMINDER: 224 equities converted to daily
Daily metrics in Performance statistics must be meaningless
Price data loaded
Cost data transaction_cost_250207.csv loaded


In [9]:
df_prices = fund.df_prices
df_prices_fees = fund._get_prices_after_fee(df_prices, fee)

In [34]:
cols = ['price', 'price_after_fees']

df_prc = df_prices[df_prices.index.is_month_end].unstack().to_frame(cols[0])
df = df_prices_fees[df_prices_fees.index.is_month_end].unstack().rename(cols[1])
df_all = df_all.join(df).rename_axis(['ticker', 'date']).dropna()
df_all

price  price_after_fees
ticker       date                                     
K55234DF2657 2021-04-30  1009.827480       1008.793994
             2021-05-31  1017.704134       1016.670649
             2021-06-30  1031.942702       1030.909216
             2021-07-31  1047.509824       1045.404287
             2021-08-31  1063.651285       1061.545748
...                              ...               ...
KR5364AH9467 2024-09-30  2706.742535       2508.757774
             2024-10-31  2755.229211       2549.018085
             2024-11-30  2823.446302       2617.235176
             2024-12-31  2939.827406       2733.616280
             2025-01-31  2983.918887       2768.790606

[17665 rows x 2 columns]

In [30]:
df_all.index.get_level_values(0).nunique()

224

In [35]:
date = df_all.index.get_level_values('date').max().strftime('%y%m%d')
date

'250131'

In [38]:
file = f'funds_monthly_{date}.csv'
path = 'pages'
df_all.to_csv(f'{path}/{file}')

## Category

In [39]:
file = 'funds_categories.csv'
path = 'data'
df_cat = pd.read_csv(f'{path}/{file}').set_index('ticker')
df_cat.columns

Index(['name', 'seller', 'account', 'manager', 'asset', 'strategy', 'region'], dtype='object')

## Bayesian

In [43]:
from pf_dash import get_hdi

file = 'fund_f3y_s3y'
path = 'data'
df_est = get_hdi(file, path, to_dict=False)
df_est.head()

fund_f3y_s3y.pkl* loaded


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
ticker,,,,,,,,,
KR5101888398,0.079,0.027,0.029,0.131,0.000,0.000,8872.0,2719.0,1.0
KR5370AP9956,0.044,0.087,-0.107,0.214,0.001,0.001,9362.0,2617.0,1.0
KR5236AS4826,0.081,0.076,-0.070,0.221,0.001,0.001,7581.0,2640.0,1.0
K55105BH1010,0.104,0.151,-0.167,0.393,0.002,0.002,8777.0,3110.0,1.0
K55301BM7640,0.208,0.130,-0.029,0.460,0.002,0.001,7049.0,2868.0,1.0


In [44]:
file = 'funds_bayesian_ret3y_250207.csv'
path = 'pages'
df_est.to_csv(f'{path}/{file}')

*get back to pages*

In [45]:
os.chdir('pages')

# Deploy

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)

## Implement

In [93]:
from dash import Dash, html, dcc, Output, Input, State
import pandas as pd
import dash_bootstrap_components as dbc
import dash_daq as daq
import json
from ddf_utils import break_line, extract_topics
from contents_info import info
from contents_topic_250204 import topics, images

external_stylesheets = [dbc.themes.CERULEAN, 
                        #dbc.themes.BOOTSTRAP,
                        dbc.icons.FONT_AWESOME,
                        dbc.icons.BOOTSTRAP]

style_heading={'color':'slategray', 'font-weight':'bold'}

default_group = '자산'
base_prc = 1000
date_format = '%Y-%m-%d'
months_in_year = 12
cols_prc = {
    'price': '수수료 적용 전', 
    'price_after_fees': '수수료 적용 후'
}
category = {
    'name': '펀드',
    'seller': '판매',
    'account': '계좌',
    'manager': '운용',
    'asset': '자산',
    'strategy': '전략',
    'region': '지역'
}

# data to import
dt = '250131'
file_prc = f'funds_monthly_{dt}.csv'
file_cat = 'funds_categories_250308.csv'
file_est = 'funds_bayesian_ret3y_250207.csv'
path = '.'

# Load data
## price
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['ticker', 'date']
)

## cateory
df_cat = pd.read_csv(f'{path}/{file_cat}', index_col=['ticker'])

## beysian stats
df_est = pd.read_csv(f'{path}/{file_est}', index_col=['ticker'])

# Preprocess data to JSON-serializable
data_cat = dict()
for col in df_cat.columns:
    data_cat[col] = df_cat[col].reset_index().groupby(col)['ticker'].apply(list).to_dict()

# additional vars
category_options = [{'label':category[x], 'value':x} for x in df_cat.columns]
category_default = 'asset'
group_default = []

### break

In [96]:
data_cat_json = json.dumps(data_cat)

app = Dash(__name__, title="달달펀드",
           external_stylesheets=external_stylesheets)

app.index_string = f"""
<!DOCTYPE html>
<html>
    <head>
        {{%metas%}}
        <title>{{%title%}}</title>
        <link rel="icon" type="image/x-icon" href="/assets/favicon.ico">
        {{%css%}}
    </head>
    <body>
        <script>
            var dataCategory = {data_cat_json};
        </script>
        {{%app_entry%}}
        {{%config%}}
        {{%scripts%}}
        {{%renderer%}}
    </body>
</html>
"""

# footer
footer = html.Footer(
    html.Small([
    html.I(className="fa-regular fa-copyright"),
    '2025 달달펀드'
]), style={'textAlign': 'right', 'margin-top': '20px'})

# info
tab_info = html.Div([
    html.P(),
    html.P('다달이 전하는 펀드 투자 정보', style=style_heading),
    html.Div(break_line(info['about'], html.P), style={'line-height': '100%'}),
    html.Div([
        dbc.Alert([
            html.I(className="bi bi-info-circle-fill me-2"),
            info['disclaimer'],
            ],
            color="info",
            className="d-flex align-items-center",
        ),
        html.P([
            html.I(className="fa-regular fa-envelope", title='문의', style={"margin-right": "10px"}),
            html.A(info['email'], href=f"mailto:{info['email']}?Subject=달달펀드:문의")
        ], style={'textAlign': 'right'})
    ], style={'fontSize': 14})
    #])
])

# tabs
tabs_contents = [
    dbc.Tab(dcc.Graph(id='price-plot'), label='가격'),
    dbc.Tab(tab_info, label='정보')
]
tabs = dbc.Tabs(tabs_contents, id='tabs')


app.layout = dbc.Container([
    html.Br(),
    dbc.Stack([
        html.Div(
            dcc.Dropdown(
                id='category-dropdown',
                options=category_options,
                value=category_default,
                clearable=False,
            ), style={'min-width':'10%', 'max-width':'100%'}
        ),
        html.Div(
            dcc.Dropdown(
                id='group-dropdown',
                #options=groups,
                value=group_default,
                multi=True,
            ), style={'min-width':'30%', 'max-width':'100%'}
        ),
        daq.BooleanSwitch(
            id='compare-boolean-switch',
            on=False
        ),
        daq.BooleanSwitch(
            id='cost-boolean-switch',
            on=False
        ),
        dcc.Clipboard(
            id='ticker-copy',
            target_id="ticker-textarea",
            style={
                "display": "inline-block",
                "fontSize": 25,
                "color": "darkgray",  # Default icon color
                "cursor": "pointer",  # Pointer cursor for better UX
                #"verticalAlign": "bottom",
            },
        ),
    ],
        #justify="start", # horizondal
        #align="center", # vertical
        direction="horizontal",
        gap=2,
        className="mb-3"
    ),
    dbc.Row(tabs),
    dbc.Row(footer),
    html.Br(),
    dcc.Textarea(
        id="ticker-textarea",
        hidden='hidden'
    ),
    dbc.Tooltip(
        '상대 비교',
        target='compare-boolean-switch',
        placement='bottom'
    ),
    dbc.Tooltip(
        '수수료 적용',
        target='cost-boolean-switch',
        placement='bottom'
    ),
    dbc.Tooltip(
        '펀드코드 복사',
        target='ticker-copy',
        placement='bottom'
    ),
    dcc.Store(id='price-data'),
    dcc.Store(id='hdi-data'),
    dcc.Location(id="url", refresh=False),  # To initialize the page
#], fluid=True)  # Full-width container
])

# update group options depending on category
app.clientside_callback(
    """
    function(category) {
        let obj = dataCategory[category];
        let groups = Object.keys(obj);
        let maxLength = 20; // Set max label length
    
        return groups.map(group => ({
            label: group.length > maxLength ? group.substring(0, maxLength) + "..." : group,
            value: group,
            title: group
        }));
    }
    """,
    Output('group-dropdown', 'options'),
    Input('category-dropdown', 'value')
)

app.clientside_callback(
    """
    function(groups, category) {
        let obj = dataCategory[category];
        let result = [];
    
        // Iterate through groups and collect values from obj
        for (let group of groups) {
            result.push(obj[group]);
        }
    
        // Convert result array to a Set (to remove duplicates) and back to an array
        result = [...new Set(result)];

        let flattenedResult = result.flat().join(\n);
        console.log(flattenedResult);
        return flattenedResult;
    }
    """,
    Output('ticker-textarea', 'value'),
    Input('group-dropdown', 'value'),
    State('category-dropdown', 'value')
)

In [97]:
if __name__ == '__main__':
    app.run_server(debug=True)

In [70]:
# Preprocess data to JSON-serializable
data_cat = dict()
for col in df_cat.columns:
    data_cat[col] = df_cat[col].reset_index().groupby(col)['ticker'].apply(list).to_dict()

# additional vars
cols = [category[x] for x in df_cat.columns]
category_options = [{'label':x, 'value':x} for x in cols]

data_cat_json = json.dumps(data_cat)

app = Dash()

app.index_string = f"""
<!DOCTYPE html>
<html>
    <head>
        {{%metas%}}
        <title>{{%title%}}</title>
        <link rel="icon" type="image/x-icon" href="/assets/favicon.ico">
        {{%css%}}
    </head>
    <body>
        <script>
            var dataCategory = {data_cat_json};
        </script>
        {{%app_entry%}}
        {{%config%}}
        {{%scripts%}}
        {{%renderer%}}
    </body>
</html>
"""


app.layout = dbc.Container([
    html.Br(),
    dbc.Stack([
        html.Div(
            dcc.Dropdown(
                id='category-dropdown',
                options=category_options,
                value=category_options[0]['value'],
                clearable=False,
            ), style={'min-width':'10%', 'max-width':'100%'}
        ),
        html.Div(
            dcc.Dropdown(
                id='group-dropdown',
                #options=groups,
                #value=default_group,
                multi=True,
            ), style={'min-width':'30%', 'max-width':'100%'}
        ),
    ]),
])

app.clientside_callback(
    """
    function(category) {
        console.log(category);
        let obj = dataCategory[category];
        console.log(obj);
        let groups = Object.keys(obj);
        return groups.map(group => ({ label: group, value: group }));
    }
    """,
    Output('group-dropdown', 'options'),
    Input('category-dropdown', 'value')
)

In [44]:
dcc.Dropdown(
    id='group-dropdown',
)

app.clientside_callback(
    """
    function(category) {
        obj = dataCategory[category]
        groups = Object.keys(obj);
        # add return of list of dropdown options
    }
    """,
    Output('group-dropdown', 'options'),
    Input('category-dropdown', 'value')
)

In [33]:
df = df_cat.iloc[:5]
pd.DataFrame().from_dict(df.to_dict())

,name,seller,account,manager,asset,strategy,region
K55101BT4402,한국투자TDF알아서2030증권투자신탁(주식혼합-재간접형)(C-Re),기업은행,퇴직연금,한국투자,혼합,TDF,글로벌
K55101BT4428,한국투자TDF알아서2035증권투자신탁(주식혼합-재간접형)(C-Re),기업은행,퇴직연금,한국투자,혼합,TDF,글로벌
K55101BT4444,한국투자TDF알아서2045증권투자신탁(주식혼합-재간접형)(C-Re),기업은행,퇴직연금,한국투자,혼합,TDF,글로벌
K55101BT7397,한국투자TDF알아서2040증권투자신탁(주식혼합-재간접형)(C-Re),기업은행,퇴직연금,한국투자,혼합,TDF,글로벌
K55101BY3944,한국투자TIF알아서평생소득증권자투자신탁(채권혼합-재간접형)(C-Re),기업은행,퇴직연금,한국투자,혼합,TIF,글로벌


In [43]:
data = df.to_dict()

In [47]:
data['strategy']

{'K55101BT4402': 'TDF',
 'K55101BT4428': 'TDF',
 'K55101BT4444': 'TDF',
 'K55101BT7397': 'TDF',
 'K55101BY3944': 'TIF'}

In [80]:
data_cat = dict()
for col in df_cat.columns:
    data_cat[col] = df_cat[col].reset_index().groupby(col)['ticker'].apply(list).to_dict()
    
data_cat

{'name': {'AB글로벌고수익증권투자신탁(채권-재간접형)종류형Ce': ['K55370BU1979', 'KR5370AP9956'],
  'AB미국그로스증권투자신탁(주식-재간접형)종류형Ce-P': ['K55370BU1755'],
  'BNK스팩&공모주30증권투자신탁 1(채권혼합)ClassC-Pe': ['K55240CP8176'],
  'IBK로우코스트TDF2030증권자투자신탁[혼합-재간접형]종류C-Re': ['K55234DF2657'],
  'IBK로우코스트TDF2035증권자투자신탁[혼합-재간접형]종류C-Re': ['K55234DF2459'],
  'IBK로우코스트TDF2040증권자투자신탁[혼합-재간접형]종류C-Re': ['K55234DF2822'],
  'IBK로우코스트TDF2045증권자투자신탁[혼합-재간접형]종류C-Re': ['K55234DF2988'],
  'IBK로우코스트TDF2050증권자투자신탁[혼합-재간접형]종류C-Re': ['K55234DF3150'],
  'IBK로우코스트TDF2055증권자투자신탁[혼합-재간접형]종류C-Re': ['K55234DF3317'],
  'IBK베트남플러스아시아증권투자신탁[주식]종류C-Re': ['K55234BY9966'],
  'IBK인컴바닐라EMP증권자투자신탁[혼합-재간접형]종류C-Pe': ['K55234DJ4477'],
  'IBK플레인바닐라EMP증권투자신탁(혼합-재간접형)C-Re': ['K55234CJ0997'],
  'IBK플레인바닐라EMP증권투자신탁[혼합-재간접형]종류C-Pe': ['K55234CJ0971'],
  'KB글로벌클린에너지수소경제증권자투자신탁(주식)C-Pe': ['K55223DF4961'],
  'KB스타미국S&P500인덱스증권자투자신탁(주식-파생형)클래스C-Pe': ['K55223DS0337'],
  'KB스타미국나스닥100인덱스증권자투자신탁(주식-파생형)(H)C-Pe': ['K55223C05630'],
  'KB스타베트남VN30인덱스증권자투자신탁(주식-파생형)C-Pe': ['K55223CN94

In [82]:
data_cat['region']['유럽']

['KR5230AR3486', 'KR5235AK9691']

In [68]:
df.reset_index().groupby(col)['ticker'].apply(list).to_dict()

{'국내': ['K55203C53681',
  'K55207BU0715',
  'K55365BT3694',
  'KR5223AE3357',
  'K55101CH6269',
  'K55102BT2108',
  'K55102BT2124',
  'K55102C83120',
  'K55102E62419',
  'K55102EF3612',
  'K55105BG6820',
  'K55105BG6945',
  'K55105BG6960',
  'K55105BH0889',
  'K55105BH0970',
  'K55107BU0237',
  'K55107BU0252',
  'K55107EA9247',
  'K55206BU2555',
  'K55206C11278',
  'K55206CQ7351',
  'K55206DZ7102',
  'K55206EA0246',
  'K55207BT6277',
  'K55207BU0699',
  'K55207D44267',
  'K55209BG6221',
  'K55209BT8966',
  'K55210C28001',
  'K55210CY1002',
  'K55210EE2770',
  'K55213DD7364',
  'K55214B25852',
  'K55214B30803',
  'K55214CD4809',
  'K55223BW3824',
  'K55229BT3864',
  'K55232BU5739',
  'K55232C99568',
  'K55236BG3001',
  'K55236BG3266',
  'K55236BG3274',
  'K55240CP8176',
  'K55301B47299',
  'K55301BU4175',
  'K55301BV3002',
  'K55301CH3249',
  'K55301CW6482',
  'K55303BT4135',
  'K55363BC7076',
  'K55363DD7958',
  'KR5101AT3522',
  'KR5101AT3563',
  'KR5101AT3571',
  'KR5101AT3589',
  'K

In [37]:
result = df.to_json()
pd.read_json(result)

## Testing

In [6]:
from dash import Dash, html, dcc, Output, Input
import pandas as pd
import dash_bootstrap_components as dbc
import dash_daq as daq
import json
from ddf_utils import break_line, extract_topics
from contents_info import info
from contents_topic_250204 import topics, images

dt = '250131'
file_prc = f'fund_monthly_{dt}.csv'
file_name = f'fund_name_{dt}.csv'
file_dst = 'fund_density_ret3y_250113.json'
path = 'history/2502'

default_group = 2030
base_prc = 1000
date_format = '%Y-%m-%d'
months_in_year = 12
cols_prc = ['수수료 적용 전', '수수료 적용 후']

# Load price data
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

# Load fund names
fund_name = pd.read_csv(f'{path}/{file_name}', dtype={'ticker': str}, index_col=[0])
fund_name = fund_name.squeeze().to_dict()

# Load density
with open(f'{path}/{file_dst}', "r") as f:
    data_density_json = f.read()  # Read raw JSON string directly

# create dropdown options of TDF groups
groups = df_prc.index.get_level_values('group').unique()
groups = [{'label': f'TDF{x}', 'value': x} for x in groups]


# Preprocess data to make it JSON-serializable and store it in a JavaScript variable
preprocessed_data = {}
df_prc.columns = cols_prc
cols = df_prc.columns
for group in groups:
    group_value = group['value']
    data = {'columns': list(cols), 'default': {}, 'compare': {}}
    start = None
    for col in cols:
        df_p = df_prc.loc[group_value, col].unstack('ticker').sort_index()
        df_p.columns = [fund_name[x] for x in df_p.columns]
        
        sr_n = df_p.apply(lambda x: x.dropna().count()) # num of months for each ticker
        df_r = (df_p.apply(lambda x: x.dropna().iloc[-1]/x.dropna().iloc[0]-1) # total return
                .to_frame('ttr').join(sr_n.rename('n'))
                .apply(lambda x: (1+x['ttr']) ** (months_in_year/x['n']) - 1, axis=1) # CAGR
                .mul(100).to_frame(col))
        
        data['default'][col] = {
            'history': df_p.round().to_dict('records'),
            'index': df_p.index.strftime(date_format).tolist(),
            'return': df_r.round(1).to_dict('records'),
            'ticker': df_r.index.tolist()
        }
        if start is None:
            start = df_p.apply(lambda x: x[x.notna()].index.min()).max()
        normalized_df = df_p.apply(lambda x: x / x.loc[start] * base_prc).loc[start:]
        
        sr_n = normalized_df.apply(lambda x: x.dropna().count()) # num of months for each ticker
        df_r_n = (normalized_df.apply(lambda x: x.dropna().iloc[-1]/x.dropna().iloc[0]-1) # total return
                  .to_frame('ttr').join(sr_n.rename('n'))
                  .apply(lambda x: (1+x['ttr']) ** (months_in_year/x['n']) - 1, axis=1) # CAGR
                  .mul(100).to_frame(col))

        data['compare'][col] = {
            'history': normalized_df.round().to_dict('records'),
            'index': normalized_df.index.strftime(date_format).tolist(),
            'return': df_r_n.round(1).to_dict('records'),
            'ticker': df_r.index.tolist()
        }
    # add fund name to ticker for ref
    tickers = df_prc.loc[group_value].index.get_level_values('ticker').unique()
    names = [f'{k}:{v}' for k,v in fund_name.items() if k in tickers]
    data['name'] = '\n'.join(names)
    # save data for group_value
    preprocessed_data[group_value] = data

In [11]:
data_grp = preprocessed_data[2030]
data_grp.keys()

dict_keys(['columns', 'default', 'compare', 'name'])

In [15]:
data_grp['default']['수수료 적용 전']['return']

[{'수수료 적용 전': 5.0},
 {'수수료 적용 전': 5.1},
 {'수수료 적용 전': 4.0},
 {'수수료 적용 전': 4.2},
 {'수수료 적용 전': 5.9},
 {'수수료 적용 전': 5.1},
 {'수수료 적용 전': 6.2},
 {'수수료 적용 전': 6.3},
 {'수수료 적용 전': 4.8},
 {'수수료 적용 전': 3.8},
 {'수수료 적용 전': 6.8},
 {'수수료 적용 전': 6.3}]

In [22]:
data_grp['columns']

['수수료 적용 전', '수수료 적용 후']

In [25]:
data_grp['default'].keys()

dict_keys(['수수료 적용 전', '수수료 적용 후'])

In [29]:
data_grp['default']['수수료 적용 전']

{'history': [{'한국투자TDF알아서2030증권투자신탁(주식혼합-재간접형)(C-Pe)': nan,
   '한국투자TDF알아서2030증권투자신탁(주식혼합-재간접형)(C-Re)': nan,
   '삼성한국형TDF2030증권투자신탁H[주식혼합-재간접형]C-Pe': nan,
   '삼성한국형TDF2030증권투자신탁H[주식혼합-재간접형]Cpe(퇴직연금)': nan,
   '키움키워드림TDF2030증권투자신탁 1[혼합-재간접형]C-P2e(퇴직연금)': nan,
   '신한마음편한TDF2030증권투자신탁[주식혼합-재간접형](종류C-re)': nan,
   '한화LIFEPLUSTDF2030증권자투자신탁(혼합-재간접형)종류C-PE(연금저축)': nan,
   '한화LIFEPLUSTDF2030증권자투자신탁(혼합-재간접형)종류C-RPE(퇴직연금)': nan,
   'KB온국민TDF2030증권투자신탁(주식혼합-재간접형)C-퇴직e': nan,
   'IBK로우코스트TDF2030증권자투자신탁[혼합-재간접형]종류C-Re': nan,
   '미래에셋전략배분TDF2030혼합자산자투자신탁 종류C-Pe': 918.0,
   '미래에셋전략배분TDF2030혼합자산자투자신탁 종류C-P2e': nan},
  {'한국투자TDF알아서2030증권투자신탁(주식혼합-재간접형)(C-Pe)': nan,
   '한국투자TDF알아서2030증권투자신탁(주식혼합-재간접형)(C-Re)': nan,
   '삼성한국형TDF2030증권투자신탁H[주식혼합-재간접형]C-Pe': nan,
   '삼성한국형TDF2030증권투자신탁H[주식혼합-재간접형]Cpe(퇴직연금)': nan,
   '키움키워드림TDF2030증권투자신탁 1[혼합-재간접형]C-P2e(퇴직연금)': nan,
   '신한마음편한TDF2030증권투자신탁[주식혼합-재간접형](종류C-re)': nan,
   '한화LIFEPLUSTDF2030증권자투자신탁(혼합-재간접형)종류C-PE(연금저축)': nan,
   '한화LIFEPLUSTDF2030증권자투자신탁(혼합-재간접형

## Samples

### Callback

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, callback, Input, Output
from io import StringIO

file_prc = 'fund_monthly_241229.csv'
path = '.'

# Load price data
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

app = Dash(__name__)
app.layout = html.Div([
    html.Button('Submit', id='submit-val', n_clicks=0),
    dcc.Graph(id='price-plot'),
    dcc.Store(id='price-data'),
])

group_value = 2030
col = 'price'
df_p = df_prc.loc[group_value, col].unstack('ticker').sort_index()

@callback(
    Output(component_id='price-data', component_property='data'),
    Input(component_id='submit-val', component_property='n_clicks')
)
def update(n_clicks):
    return df_p.to_json(date_format='iso', orient='split')


@callback(
    Output(component_id='price-plot', component_property='figure'),
    Input(component_id='price-data', component_property='data')
)
def plot(data):
    # Use StringIO to wrap the string data
    data_io = StringIO(data)
    df = pd.read_json(data_io, orient='split')
    return px.line(df)

if __name__ == '__main__':
    app.run_server(debug=True)

### Clientside

In [ ]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc
import json

file_prc = 'fund_monthly_241229.csv'
path = '.'

# Load price data
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

app = Dash(__name__)

group_value = 2030
col = 'price'
df_p = df_prc.loc[group_value, col].unstack('ticker').sort_index()

# Preprocess the data and serialize it to JSON
preprocessed_data = df_p.to_json(date_format='iso', orient='split')

app.layout = html.Div([
    dcc.Graph(id='price-plot'),
    # Embed the preprocessed data as a hidden div or dcc.Store
    dcc.Store(id='price-data', data=preprocessed_data),
])

# Clientside callback for plotting
app.clientside_callback(
    """
    function(data) {
        if (!data) {
            return {'data': [], 'layout': {}};
        }
        // Parse the JSON data
        const df = JSON.parse(data);
        const dates = df.index;
        const columns = df.columns;

        const traces = columns.map((col, i) => ({
            x: dates,
            y: df.data.map(row => row[i]),
            mode: 'lines',
            name: col
        }));

        return {
            data: traces,
            layout: {
                title: 'Price Plot',
                xaxis: {title: 'Date'},
                yaxis: {title: 'Price'}
            }
        };
    }
    """,
    Output('price-plot', 'figure'),
    Input('price-data', 'data')
)

if __name__ == '__main__':
    app.run_server(debug=True)


### Mobile

In [ ]:
import pandas as pd
from dash import Dash, html, dcc
import dash_bootstrap_components as dbc
import json

file_prc = 'fund_monthly_241229.csv'
path = '.'

# Load price data
df_prc = pd.read_csv(
    f'{path}/{file_prc}',
    parse_dates=['date'],
    dtype={'ticker': str},
    index_col=['group', 'ticker', 'date']
)

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

group_value = 2030
col = 'price'
df_p = df_prc.loc[group_value, col].unstack('ticker').sort_index()

# Preprocess the data and serialize it to JSON
preprocessed_data = df_p.to_json(date_format='iso', orient='split')

app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                html.Button('Submit', id='submit-val', n_clicks=0),
                width={"size": 6, "offset": 3},  # Centered on the screen
                className="text-center"
            )
        ),
        dbc.Row(
            dbc.Col(
                dcc.Graph(id='price-plot'),
                width=12  # Full-width on all screen sizes
            ),
            className="mt-4"  # Add margin-top for spacing
        ),
        # Store the preprocessed data
        dcc.Store(id='price-data', data=preprocessed_data),
    ],
    fluid=True  # Full-width container
)

# Clientside callback for plotting
app.clientside_callback(
    """
    function(data) {
        if (!data) {
            return {'data': [], 'layout': {}};
        }
        // Parse the JSON data
        const df = JSON.parse(data);
        const dates = df.index;
        const columns = df.columns;

        const traces = columns.map((col, i) => ({
            x: dates,
            y: df.data.map(row => row[i]),
            mode: 'lines',
            name: col
        }));

        return {
            data: traces,
            layout: {
                title: 'Price Plot',
                xaxis: {title: 'Date'},
                yaxis: {title: 'Price'},
                margin: {l: 40, r: 10, t: 40, b: 40},
                responsive: true
            }
        };
    }
    """,
    Output('price-plot', 'figure'),
    Input('price-data', 'data')
)

if __name__ == '__main__':
    app.run_server(debug=True)


## testing

## testing

## testing

In [ ]:
import pandas as pd

In [ ]:
file = 'fund_name_241230.csv'
path = '.'

fund_name = pd.read_csv(f'{path}/{file}', dtype={'ticker': str}, index_col=[0])
fund_name = fund_name.iloc[:,0].to_dict()
fund_name

In [ ]:
import re


m = re.search(r'\d{4}', s)
func = lambda s: [s if x is None else s[:x.end()] for x in [re.search(r'\d{4}', s)]][0]

In [ ]:
s = '신한마음편한TDF2040증권투자신탁[주식혼합-재간접형](종류C-re)'
#s = '신한마음편한TDF증권투자신탁[주식혼합-재간접형](종류C-re)'

func = lambda s: s[:m.end()] if (m := re.search(r'\d{4}', s)) else s

func(s)

In [ ]:
#i = 2050
i = 2055
data = preprocessed_data[i]
#data.keys()

col = '수수료 적용 전'
d = data['default'][col]['history']
i = data['default'][col]['index']
df = pd.DataFrame(d, index=i)

import plotly.express as px
px.line(df)

In [ ]:
import plotly.graph_objects as go

i = 2050
data = preprocessed_data[i]
#data.keys()

col = '수수료 적용 전'
d = data['default'][col]['history']
i = data['default'][col]['index']

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=i, y=d,
                    mode='lines',
                    #name='ticker'
                        )
             )
fig.show()

In [ ]:
# You can use Plotly's `plot` function to visualize the result
import plotly.offline as pyo
pyo.plot(result)

In [ ]:
d

In [ ]:
"""
            return {
                x: dat.index,        // Dates from the index
                y: yValues,          // Price history for each ticker
                type: 'scatter',
                mode: 'lines',
                name: ticker,        // Ticker as the legend name
                //color: ticker
            };
            """

In [ ]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np
np.random.seed(1)

N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N) + 5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N) - 5

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='markers',
                    name='markers'))


"""
            return {
                x: dat.index,        // Dates from the index
                y: yValues,          // Price history for each ticker
                type: 'scatter',
                mode: 'lines',
                name: ticker,        // Ticker as the legend name
                //color: ticker
            };
            """

In [ ]:
import plotly.graph_objects as go
import pandas as pd

data= pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/2014_usa_states.csv")

fig = go.Figure(data=go.Scatter(x=data['Postal'],
                                y=data['Population'],
                                mode='markers',
                                marker_color=data['Population'],
                                text=data['State'])) # hover text goes here

fig.update_layout(title=dict(text='Population of USA States'))
fig.show()

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("country=='Canada'")
fig = px.line(df, x="year", y="lifeExp", title='Life expectancy in Canada', color='country')
fig.show()

In [ ]:
df = px.data.gapminder().query("continent=='Oceania'")
fig = px.line(df, x="year", y="lifeExp", color='country')
fig.show()